# Cloud Data Transfer Speeds Benchmarking Workflow

Add overview of workflow

## Step 0: Load Required Setup Packages & Classes

Installs required workflow setup packages and calls UI generation script. If one or more of the packages don't exist in your `base` environment, they will install for you. Note that if installation is required, this cell will take a few minutes to complete execution.

In [1]:
import os
import json
import sys

print('Checking conda environment for UI depedencies...')
os.system("bash " + os.getcwd() + "/jupyter-helpers/install_ui_packages.sh")
print('Done.')

sys.path.insert(0, os.getcwd() + '/jupyter-helpers')
import ui_helpers as ui

Checking conda environment for UI depedencies...
Done.


## Step 1: Define Workflow Inputs

Run the following cells to generate interactive widgets allowing you to enter all workflow inputs. **All inputs must be filled out to proceed with the benchmarking process.**

### Cloud Resources

#### Compute Resources

Before defining anything else, the resources you intent to use with the benchmarking must be defined. Currently, only resources defined in the Parallel Works platform may be used. Also of note are options that will be passed to Dask: you have full control over how many cores and memory you want each worker from your cluster to use, as well as how many nodes you want to be active at a single time.

In particular, these options are included so that you can form fair comparisons between different cloud service providers (CSPs). Generally, different CSPs won't have worker nodes with the exact same specs, and in order to achieve a fair comparison between two CSPs one cluster will have to limited to not exceed to the computational power of the other.

<div class="alert alert-block alert-info">
For resources controlled from the Parallel Works platform, the <code>Resource name</code> box should be populated with the name found on the <b>RESOURCES</b>  tab.
    </div>

In [2]:
resource = ui.resourceWidgets()
resource.display()
resources = resource.processInput()
print(f'Your resource inputs:\n {resources}')

Accordion(children=(VBox(children=(HBox(children=(Label(value='Resource Name: '), Text(value='', placeholder='…

Button(description='Submit', style=ButtonStyle())


-----------------------------------------------------------------------------
If you wish to change information about cloud resources, run this cell again.

Your resource inputs:
 [{'Name': 'gcptestnew', 'Controller': 'PW', 'CSP': 'GCP', 'Dask': {'Scheduler': 'SLURM', 'Partition': 'compute', 'CPUs': 2, 'Memory': 16.0}}]


#### Object Stores

This set of inputs is where you enter the cloud object store Universal Resource Identifiers (URIs). Both public and private buckets are supported. For the latter, ensure that you have access credentials with both read and write permissions, as format conversions will need to be made during the benchmarking process.

In [3]:
store = ui.storageWidgets()
store.display()
storage = store.processInput()
print(f'Your storage inputs:\n {storage}')

Accordion(children=(VBox(children=(HBox(children=(Label(value='Storage URI/Mount Path: '), Text(value='', plac…

Button(description='Submit', style=ButtonStyle())


--------------------------------------------------------------------------------------
If you wish to change information about cloud storage locations, run this cell again.

Your storage inputs:
 [{'Path': 'gs://cloud-data-benchmarks', 'Type': 'Private', 'CSP': 'GCP', 'Credentials': './.cloud-data-benchmarks.json'}]


### Datasets

#### User-Supplied Datasets

Below you can specify datasets that you want to be tested in the benchmarking. You can either enter single files or multiple files that belong to a single dataset, but that dataset match at least one of the supported formats.

In the "Full URI/Path of Data section", single files should be entered as `<URI>://bucket-name/path/to/file.extension`. Datasets containing multiple files may be entered as a globstring: `<URI>://bucket-name/path/to/files/*`

<div class="alert alert-block alert-info">
In the widget generated below, leaving the box unchecked means that you do not wish to input any of your own files. Upon pressing the submit button, no inputs will be recorded (even if you filled out fields)
    </div>

In [4]:
userdata = ui.userdataWidgets(storage=storage)
userdata.display()
user_files = userdata.processInput()
print(f'Your data inputs:\n {user_files}')

Checkbox(value=False, description='Provide datasets to workflow?')

Accordion(children=(VBox(children=(HBox(children=(Label(value='Data Format'), Dropdown(options=('CSV', 'NetCDF…

Button(description='Submit', style=ButtonStyle())


---------------------------------------------------------------------------
If you wish to change information about your input data, run this cell again.

Your data inputs:
 [{'Format': 'NetCDF4', 'Path': 'gs://cloud-data-benchmarks/ETOPO1_Ice_g_gmt4.nc', 'Type': 'Private', 'CSP': 'GCP', 'Credentials': './.cloud-data-benchmarks.json'}]


#### Randomly-Generated Datasets

Another option to supply data to the benchmarking is to create randomly-generated datasets. These sets can be as large as you want (as they are written in parallel), and provide a great option if you are new to the world of cloud-native data formats. There are currently supported randomly-generated data formats: CSV and NetCDF4. Since NetCDF4 is a gridded data format, options to customize the types and numbers of dimensions are included.

In [5]:
randgen = ui.randgenWidgets(resources=resources)
randgen.display()
randfiles = randgen.processInput()
print(f'Your randomly-generated file options:\n {randfiles}')

Accordion(children=(HBox(children=(Label(value='Resource to write random files with: '), Dropdown(options=('gc…

Button(description='Submit', style=ButtonStyle())


-------------------------------------------------------------------------------
If you wish to change the randomly generated file options, run this cell again.

Your randomly-generated file options:
 [{'Format': 'CSV', 'Generate': True, 'SizeGB': 5.0}, {'Format': 'NetCDF4', 'Generate': True, 'SizeGB': 100.0, 'Data Variables': 2.0, 'Float Coords': 3.0, 'Time Coords': 1.0}, {'Resource': 'gcptestnew'}]


### Cloud-Native Format Conversion Options

You also have the options of setting which chunksizes and compression algorithms you'd like to use in the benchmarking.

## Step 2: Notebook Setup

Executing the following cell will write all of your inputs to `benchmark_info.json`, install miniconda3 and the "cloud-data" Python environment to all resources, and write randomly-generated files to all cloud storage locations (if any files were specified). If writing randomly-generated files, especially large ones, the execution of this cell may take a while.

<div class="alert alert-block alert-info">
While randomly-generated files are written in parallel by default, if you wish to speed up the execution of this cell, consider creating/choosing a resource with more powerful worker nodes.
    </div>

In [6]:
print('Setting up workflow...')

user_input = json.dumps({"RESOURCES" : resources,
             "STORAGE" : storage,
             "USERFILES" : user_files,
             "RANDFILES" : randfiles})

with open('benchmark_info.json', 'w') as outfile:
    outfile.write(user_input)

#os.system("bash workflow_notebook_setup.sh")

print('Workflow setup complete.')

Setting up workflow...
Workflow setup complete.


## Step 3: Run Benchmarking

In [ ]:
os.system('bash benchmarks-core/convert-data/convert_data.sh')

## Step 4: Visualize Results